## Kabul NO₂ for January 2024 (Sentinel-5P)

In this notebook, I use the Sentinel-5P satellite (TROPOMI instrument) to compute the monthly average concentration of tropospheric NO₂ over Kabul for January 2024. NO₂ is a key indicator of traffic and combustion-related air pollution. 

The map below shows spatial variation in NO₂ levels across the Kabul region, using a simple bounding box around the city center.


In [1]:
import ee
import geemap
import datetime as dt

ee.Initialize()

print("Earth Engine initialized successfully.")

Earth Engine initialized successfully.


In [2]:
kabul_lat = (34.5553)  # Kabul coordinates
kabul_lon = (69.2075)

def make_city_bbox(lon, lat, half_deg=0.25):
    """
    Create a rectangular bounding box around a city given its longitude and latitude.
    half_deg = how big the box is in degrees (0.25 = ~25-30 kn each way)
    """
    return ee.Geometry.Rectangle([
        lon - half_deg,
        lat - half_deg,
        lon + half_deg,
        lat + half_deg
    ])

kabul_geom = make_city_bbox(kabul_lon, kabul_lat, half_deg=0.25)  # ~25-30 km box around Kabul
kabul_geom

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              68.9575,
              34.8053
            ],
            [
              68.9575,
              34.3053
            ],
            [
              69.4575,
              34.3053
            ],
            [
              69.4575,
              34.8053
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [ ]:
# load Sentinel-5P NO2 and filter for Jan 2024

# Sentinel-5P NO2 collection
no2_collection = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2") \
    .select("tropospheric_NO2_column_number_density")

# define time range: Jan 2024
start_date= "2024-01-01"
end_date= "2024-02-01"

no2_kabul_jan2024=(
    no2_collection.filterDate(start_date, end_date)
    .filterBounds(kabul_geom)
    .mean() #avg over the month
)

no2_kabul_jan2024

In [ ]:
# visualize NO2 over Kabul on an interactive map

# clip the monthly NO2 image to the Kabul bounding box
no2_kabul_jan2024_clipped = no2_kabul_jan2024.clip(kabul_geom)

# visualization parameters for NO2
no2_vis_params = {
    "min": 0,
    "max": 0.0002,
    "palette": ["blue", "green", "yellow", "red"],
}

# clip the monthly NO2 image to the Kabul area
no2_kabul_jan2024_clipped = no2_kabul_jan2024.clip(kabul_geom)

# create map centered on Kabul
m = geemap.Map(center=[kabul_lat, kabul_lon], zoom=10)

# add the clipped NO2 layer only
m.addLayer(no2_kabul_jan2024_clipped, no2_vis_params, "Kabul NO2,  Jan 2024")

m

Map(center=[34.5553, 69.2075], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…